In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
PATH = 'C:/Users/EunJin/Desktop/dacon/LG AI/'

# 함수쓰기 전에 꼭 
# train_err = pd.read_csv(PATH+'train_err_data.csv')
# train_quality = pd.read_csv(PATH+'train_quality_data.csv')
# 로 데이터 다시 불러서 함수 써주세요!!

In [2]:
def make_df(train_err, quality, select=0):
    
    # select=0 : errday, quality, errtype, errcode, model, fwver
    # select=1 : errday, quality, errtype*errcode, model, fwver
    # select=2 : errday, quality, errtype*errcode*fwver, model    
        
    
    #train_err  = pd.read_csv(PATH+'train_err_data.csv')
    train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
    train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
    train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

    train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
    train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
    train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))
    train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))
    
    
    
    ###err_day
    pivot_err_day = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
    df_errday = pd.DataFrame(pivot_err_day.to_records())

    df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 

    for i in list(range(1,33)) :
            df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

   
    
    ###quality
    
    #quality = pd.read_csv(PATH+'train_quality_data.csv')
    quality.fillna(0, inplace = True) #quality_5변수에 NA값이 있음

    cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
              'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
    quality[cols] = quality[cols].replace({',':''}, regex= True) 
    quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

    minusQ = []
    for i in range(0,len(quality)): 
        if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
        #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
          minusQ.append(-1)
        else: minusQ.append(0)

    quality['minusQ'] = minusQ 


    def func(x):
        d = {}
        d['quality_0'] = x['quality_0'].max()
        d['quality_1'] = x['quality_1'].max()
        d['quality_2'] = x['quality_2'].max()
        d['quality_3'] = x['quality_3'].max()
        d['quality_4'] = x['quality_4'].max()
        d['quality_5'] = x['quality_5'].max()
        d['quality_6'] = x['quality_6'].max()
        d['quality_7'] = x['quality_7'].max()
        d['quality_8'] = x['quality_8'].max()
        d['quality_9'] = x['quality_9'].max()
        d['quality_10'] = x['quality_10'].max()
        d['quality_11'] = x['quality_11'].max()
        d['quality_12'] = x['quality_12'].max()
        d['minusQ'] = x['minusQ'].sum()
        
        return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                      'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

    quality5 = quality.groupby(['user_id','time']).apply(func)

    quality5 = quality5.groupby(['user_id']).sum()
    quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 

  

    ###err_type
    pivot_err_type = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
    df_errtype = pd.DataFrame(pivot_err_type.to_records())
    df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 



    ###errcode
    pivot_errcode = pd.pivot_table(train_err, index = ['user_id'],columns = 'errcode', values = ['time'],aggfunc='count',fill_value=0)
    df_errcode = pd.DataFrame(pivot_errcode.to_records())

    df_errcode = df_errcode.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errcode_"))



    ### model
    pivot_model = pd.pivot_table(train_err, index = ['user_id'],columns = 'model_nm', values = ['time'],aggfunc='count',fill_value=0)
    df_model = pd.DataFrame(pivot_model.to_records())

    df_model = df_model.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time",""))

    

    ###fwver
    pivot_fwver = pd.pivot_table(train_err, index = ['user_id'],columns = 'fwver', values = ['time'],aggfunc='count',fill_value=0)
    df_fwver = pd.DataFrame(pivot_fwver.to_records())

    df_fwver = df_fwver.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","fwver"))
    
    
    
    ###errtype*errcode
    temp1 = train_err
    temp1['errtype_code'] = temp1['errtype'].astype(str)+'_'+temp1['errcode']
    
    pivot_errtype_code = pd.pivot_table(temp1, index = ['user_id'],columns = 'errtype_code', values = ['time'],aggfunc='count',fill_value=0)
    df_errtype_code = pd.DataFrame(pivot_errtype_code.to_records())

    df_errtype_code = df_errtype_code.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype_code"))
        
    
    ###errtype*errcode*fwver
    temp1['errtype_code_fwver'] = temp1['errtype'].astype(str)+'_'+temp1['errcode']+'_'+temp1['fwver']
    
    pivot_errtype_code_fwver = pd.pivot_table(temp1, index = ['user_id'],columns = 'errtype_code_fwver', values = ['time'],aggfunc='count',fill_value=0)
    df_errtype_code_fwver = pd.DataFrame(pivot_errtype_code_fwver.to_records())

    df_errtype_code_fwver = df_errtype_code_fwver.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype_code_fwver"))
            
    
    
    
    # select=0 : errday, quality, errtype, errcode, model, fwver
    # select=1 : errday, quality, errtype*errcode, model, fwver
    # select=2 : errday, quality, errtype*errcode*fwver, model    
    
    
    
    
    
    ####merge
    
    if select==0:
        df_result = pd.merge(df_errday, quality5, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errtype, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errcode, on='user_id', how='left')
        df_result = pd.merge(df_result, df_model, on='user_id', how='left')
        df_result = pd.merge(df_result, df_fwver, on='user_id', how='left')
        
    
    elif select==1:
        df_result = pd.merge(df_errday, quality5, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errtype_code, on='user_id', how='left')
        df_result = pd.merge(df_result, df_model, on='user_id', how='left')
        df_result = pd.merge(df_result, df_fwver, on='user_id', how='left')
        
    else :
        df_result = pd.merge(df_errday, quality5, on='user_id', how='left')
        df_result = pd.merge(df_result, df_errtype_code_fwver, on='user_id', how='left')
        df_result = pd.merge(df_result, df_model, on='user_id', how='left')
        

    df_result.fillna(0, inplace = True)

    return df_result

In [ ]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')

X_train0 = make_df(train_err, train_quality, select=0)

display(X_train1.head())
print(X_train1.columns)
print(X_train0.shape)